In [44]:
import os

notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '../'))
os.chdir(project_root)

import sys
sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset

from src.config import CCY_PAIRS, START_DATE, END_DATE
from src.data_utils import get_fx_data, compute_log_ret, to_binary, to_float, get_min_max_info
from src.models.gan import GAN, preprocess_data

In [46]:
df = get_fx_data(CCY_PAIRS, START_DATE, END_DATE, df=True)
log_ret_df = compute_log_ret(df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [47]:
log_ret_df[log_ret_df.columns[9:]]

,EURUSD_log_ret,GBPUSD_log_ret,NOKSEK_log_ret,USDBRL_log_ret,AUDUSD_log_ret,USDCAD_log_ret,EURPLN_log_ret,EURDKK_log_ret,USDCNY_log_ret
Date,,,,,,,,,
2010-01-05,-0.004031,-0.007657,0.006299,0.010417,-0.000730,-0.001442,0.002862,0.000000,0.000015
2010-01-06,0.002647,0.002065,0.003857,-0.003171,0.009436,-0.007338,-0.000757,-0.000134,0.000191
2010-01-07,-0.005989,-0.005625,-0.000722,0.004265,-0.002941,0.002033,0.004244,0.000027,-0.000073
2010-01-08,0.006478,0.005641,-0.004908,-0.003976,0.007924,-0.004070,-0.011235,-0.000081,-0.000044
2010-01-11,0.006927,0.005527,0.011308,-0.000578,0.005008,0.003877,0.000861,0.000054,-0.000147
...,...,...,...,...,...,...,...,...,...
2023-12-25,0.001851,-0.000630,-0.012391,-0.002452,0.001762,-0.002042,0.001951,-0.000129,-0.007763
2023-12-26,-0.000573,0.002195,0.005943,-0.000021,-0.000408,-0.000513,0.000455,-0.000150,0.007342
2023-12-27,0.002063,0.001755,0.008249,-0.011617,0.002611,-0.004144,-0.001957,0.000121,0.000967


In [48]:
log_ret_tensor = preprocess_data(log_ret_df[log_ret_df.columns[9:]])

In [53]:
# Define parameters
latent_dim = 100
output_dim = log_ret_tensor.shape[1]
epochs = 2000
batch_size = 128

In [54]:
data_loader = DataLoader(TensorDataset(log_ret_tensor), batch_size=batch_size, shuffle=True)

In [55]:
gan = GAN(latent_dim, output_dim)

In [56]:
gan.fit(data_loader, epochs, batch_size)

Epoch [0/2000], Batch [28/29], D Loss: 1.3945, G Loss: 0.7175
Epoch [1/2000], Batch [28/29], D Loss: 1.3856, G Loss: 0.7063
Epoch [2/2000], Batch [28/29], D Loss: 1.3800, G Loss: 0.6989
Epoch [100/2000], Batch [28/29], D Loss: 1.3864, G Loss: 0.6933
Epoch [200/2000], Batch [28/29], D Loss: 1.3865, G Loss: 0.6929
Epoch [300/2000], Batch [28/29], D Loss: 1.3866, G Loss: 0.6932
Epoch [400/2000], Batch [28/29], D Loss: 1.3860, G Loss: 0.6933
Epoch [500/2000], Batch [28/29], D Loss: 1.3866, G Loss: 0.6932
Epoch [600/2000], Batch [28/29], D Loss: 1.3861, G Loss: 0.6932
Epoch [700/2000], Batch [28/29], D Loss: 1.3862, G Loss: 0.6932
Epoch [800/2000], Batch [28/29], D Loss: 1.3866, G Loss: 0.6931
Epoch [900/2000], Batch [28/29], D Loss: 1.3862, G Loss: 0.6933
Epoch [1000/2000], Batch [28/29], D Loss: 1.3859, G Loss: 0.6934
Epoch [1100/2000], Batch [28/29], D Loss: 1.3862, G Loss: 0.6933
Epoch [1200/2000], Batch [28/29], D Loss: 1.3863, G Loss: 0.6933
Epoch [1300/2000], Batch [28/29], D Loss: 1

In [57]:
generated_samples = gan.generate_samples(1000)

In [58]:
pairs=df.columns

synth = pd.DataFrame(generated_samples)
synth.columns = pairs


In [59]:
synth

,EURUSD,GBPUSD,NOKSEK,USDBRL,AUDUSD,USDCAD,EURPLN,EURDKK,USDCNY
0,-0.000714,-0.005598,-0.008424,0.004150,-0.005089,0.000582,0.000030,-0.000201,0.000386
1,-0.002196,-0.006107,-0.008034,0.003650,-0.009123,0.001346,0.002159,-0.000232,-0.000082
2,0.001133,-0.008640,-0.007774,0.002959,-0.004748,0.000722,-0.002069,0.000203,0.001040
3,-0.000830,-0.008237,-0.009089,0.004788,-0.006563,0.001768,0.000034,0.000191,0.001164
4,0.001163,-0.008752,-0.007972,0.003208,-0.004952,0.000822,-0.002162,0.000199,0.001046
...,...,...,...,...,...,...,...,...,...
995,0.001482,-0.007543,-0.006880,0.000685,-0.004501,0.000676,-0.002300,0.000075,0.000616
996,0.000420,-0.007904,-0.005378,-0.000568,-0.002953,-0.000273,-0.000973,0.000222,0.000734
997,0.004498,-0.015354,-0.011097,-0.006143,-0.001158,-0.006014,-0.012215,-0.001934,-0.000322
998,0.002679,-0.002918,-0.005275,-0.002342,-0.002037,0.000121,-0.001646,-0.000220,-0.000763


In [60]:
min_max

{'EURUSD': [-0.028144080771163097, 0.031262324758573316],
 'GBPUSD': [-0.07908477228809001, 0.03030790933040477],
 'NOKSEK': [-0.03647251810176716, 0.03916921475768578],
 'USDBRL': [-0.06380101045012901, 0.0724972866792708],
 'AUDUSD': [-0.041266121501451584, 0.028532926147473537],
 'USDCAD': [-0.0319381679597881, 0.029692673692160075],
 'EURPLN': [-0.03197035026320121, 0.03479261479555058],
 'EURDKK': [-0.00266741186497614, 0.002680883743968607],
 'USDCNY': [-0.09290397407449433, 0.09832922673313901]}

In [61]:
from src.data_utils import get_min_max_info, to_float, get_latest_value, compute_log_ret

min_max = get_min_max_info(log_ret_df)

In [62]:
res = np.exp(synth.astype(float)).cumprod() # Cumulated returns

fx0 = get_latest_value(log_ret_df)
for column, factor in fx0.items():
    res[column] *= factor


In [63]:
res

,EURUSD,GBPUSD,NOKSEK,USDBRL,AUDUSD,USDCAD,EURPLN,EURDKK,USDCNY
0,1.106030,1.266290,0.969469,4.870372,0.679165,1.323730,4.333681,7.451275,7.109740
1,1.103603,1.258580,0.961711,4.888179,0.672997,1.325514,4.343047,7.449546,7.109160
2,1.104854,1.247753,0.954264,4.902666,0.669809,1.326472,4.334072,7.451055,7.116557
3,1.103938,1.237518,0.945630,4.926197,0.665427,1.328819,4.334221,7.452479,7.124843
4,1.105223,1.226735,0.938121,4.942025,0.662140,1.329912,4.324860,7.453962,7.132301
...,...,...,...,...,...,...,...,...,...
995,40.201159,0.018516,0.003610,0.037969,0.917578,0.074743,0.075047,3.276555,2.050426
996,40.218058,0.018370,0.003591,0.037947,0.914872,0.074722,0.074974,3.277284,2.051933
997,40.399352,0.018090,0.003551,0.037715,0.913814,0.074274,0.074064,3.270952,2.051273
998,40.507709,0.018038,0.003532,0.037627,0.911954,0.074283,0.073942,3.270231,2.049708
